###  **Candidate number: 41830**

# Assignment week 5 (summative)

In this assignment you will use the Openweathermap Current Whether Data API to access weather information for cities in the UK. You need to sign up for an account with Openweathermap to get a Key. You will get 1,000 API calls per day for free. https://openweathermap.org/api

#### Q1. Get a list of all cities in the UK (excluding Crown Dependencies and Overseas Territories). (15 points)
Hint: To get the name of all cities in UK, consider scraping this website https://www.gov.uk/government/publications/list-of-cities/list-of-cities-html.

In [102]:
import requests
res = requests.get("https://www.gov.uk/government/publications/list-of-cities/list-of-cities-html")
soup = BeautifulSoup(res.content,'lxml')
lists = soup.find_all('li')
char = []
ind = []

for dt in lists:
    row = []
    chars = dt.text.split(': ')
    ind.append(chars)
ind
# Exclude all cities that are not in UK
fentry = ind.index(['Bath'])
lentry = ind.index(['Wrexham'])
cities = ind[fentry:lentry+1] #to include last entry
cities = [item[0] for item in cities] 

cleaned_cities = []
for item in cities:
    clean_city = item.replace('*', '') 
    cleaned_cities.append(clean_city)

#Based on the email, im chosing to exclude both Bangor
cleaned_cities.remove('Bangor')

print(cleaned_cities)

['Bath', 'Birmingham', 'Bradford', 'Brighton & Hove', 'Bristol', 'Cambridge', 'Canterbury', 'Carlisle', 'Chelmsford', 'Chester', 'Chichester', 'Colchester', 'Coventry', 'Derby', 'Doncaster', 'Durham', 'Ely', 'Exeter', 'Gloucester', 'Hereford', 'Kingston-upon-Hull', 'Lancaster', 'Leeds', 'Leicester', 'Lichfield', 'Lincoln', 'Liverpool', 'London', 'Manchester', 'Milton Keynes', 'Newcastle-upon-Tyne', 'Norwich', 'Nottingham', 'Oxford', 'Peterborough', 'Plymouth', 'Portsmouth', 'Preston', 'Ripon', 'Salford', 'Salisbury', 'Sheffield', 'Southampton', 'Southend-on-Sea', 'St Albans', 'Stoke on Trent', 'Sunderland', 'Truro', 'Wakefield', 'Wells', 'Westminster', 'Winchester', 'Wolverhampton', 'Worcester', 'York', 'Armagh', 'Belfast', 'Lisburn', 'Londonderry', 'Newry', 'Aberdeen', 'Dundee', 'Dunfermline', 'Edinburgh', 'Glasgow', 'Inverness', 'Perth', 'Stirling', 'Bangor', 'Cardiff', 'Newport', 'St Asaph', 'St Davids', 'Swansea', 'Wrexham']


#### Q2. Use the Openweathermap Current Whether Data API to access the following information for each city: **main.temp, wind.speed, rain (1h) and Time of data calculation (dt)**. Use default units for these variables. Add the city name, temperature, wind, rain and dt information to a DataFrame as separate columns. If one of these attributes is missing, replace it with NONE. (25 points)
Hint: You may find it easier to create a dictionary with cities as keys and temperature, wind, rain and date as values first and then convert it into a DataFrame.

In [103]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import json
api_key = "640ef4eecc6d375e896d081557ec3408"
#Initialize
mtepm = []
wspee = []
rain = []
dtime = []

for item in cleaned_cities:
    
    query = 'q='+item
    res=requests.get('http://api.openweathermap.org/data/2.5/weather?'+query+'&APPID='+api_key)
    data = res.json()

    mt = data.get('main', {}).get('temp', 'none')
    mtepm.append(mt)
    ws = data.get('wind', {}).get('speed', 'none')
    wspee.append(ws)
    ra = data.get('rain', {}).get('1h', 'none') 
    rain.append(ra)
    dt = data.get('dt', "none") 
    dtime.append(dt)
    
data = {
    'City name': cleaned_cities,
    'Temperature': mtepm,
    'Wind' : wspee,
    'Rain' : rain,
    'dt' : dtime
}
df = pd.DataFrame(data)
df.head()


,City name,Temperature,Wind,Rain,dt
0,Bath,285.85,1.83,none,1730898953
1,Birmingham,293.86,2.34,none,1730899221
2,Bradford,287.03,1.54,none,1730898956
3,Brighton & Hove,284.08,1.54,none,1730899039
4,Bristol,291.7,4.63,none,1730899065


#### Q3. The Time of data calculation (dt) is in Unix Timestamp format. Convert it to standard UTC (``dd/mm/yy @ HH:MM:SS``). Print out the updated DataFrame. (5 points)
Hint: You may use ``datetime.fromtimestamp`` to convert dt.

In [104]:
import pandas as pd
from datetime import datetime

# I had to define a function bc the none values where causing an error
def safe_fromtimestamp(x):
    try:
        #convert to time
        return datetime.fromtimestamp(int(x))
    except (ValueError, TypeError):
        # return none if conversion fails
        return 'none'

df['dt'] = df['dt'].apply(safe_fromtimestamp)
print(df)

          City name Temperature  Wind  Rain                   dt
0              Bath      285.85  1.83  none  2024-11-06 13:15:53
1        Birmingham      293.86  2.34  none  2024-11-06 13:20:21
2          Bradford      287.03  1.54  none  2024-11-06 13:15:56
3   Brighton & Hove      284.08  1.54  none  2024-11-06 13:17:19
4           Bristol       291.7  4.63  none  2024-11-06 13:17:45
..              ...         ...   ...   ...                  ...
70          Newport      286.14  0.85  none  2024-11-06 13:15:54
71         St Asaph       288.5  1.37  none  2024-11-06 13:20:25
72        St Davids      286.46  7.29  none  2024-11-06 13:17:22
73          Swansea      287.26  2.68  none  2024-11-06 13:18:08
74          Wrexham      287.11  4.63  none  2024-11-06 13:20:25

[75 rows x 5 columns]


#### Q4. The default temperature units in Openweathermap are Kelvin. Convert it to Celsius. Print out the DataFrame. (5 points)
Hint: You may use the API to access the temperature in Celsius or use the convertion formula to convert the Kelvin values in your DataFrame. 

In [105]:
import pandas as pd
from datetime import datetime

# Idem for dt, I had to define a function
def safe_kelvintocel(x):
    try:
        if int(x) > 100: # T>100 means that the temp was in Kelvin
            return int(x)-273
        else:
            return int(x)
    
    except (ValueError, TypeError):
        # return none if conversion fails
        return 'none'

df['Temperature'] = df['Temperature'].apply(safe_kelvintocel)
print(df)

          City name Temperature  Wind  Rain                   dt
0              Bath          12  1.83  none  2024-11-06 13:15:53
1        Birmingham          20  2.34  none  2024-11-06 13:20:21
2          Bradford          14  1.54  none  2024-11-06 13:15:56
3   Brighton & Hove          11  1.54  none  2024-11-06 13:17:19
4           Bristol          18  4.63  none  2024-11-06 13:17:45
..              ...         ...   ...   ...                  ...
70          Newport          13  0.85  none  2024-11-06 13:15:54
71         St Asaph          15  1.37  none  2024-11-06 13:20:25
72        St Davids          13  7.29  none  2024-11-06 13:17:22
73          Swansea          14  2.68  none  2024-11-06 13:18:08
74          Wrexham          14  4.63  none  2024-11-06 13:20:25

[75 rows x 5 columns]
